In [1]:
import numpy as np
from torch import tensor, no_grad, Tensor
import torch
import json

from thesis_v2.models.maskcnn_polished_with_rcnn_k_bl.builder import (
    gen_maskcnn_polished_with_rcnn_k_bl, load_modules
)

load_modules()

In [2]:
np.random.seed(0)
random_img = np.random.random_sample((10, 50, 50, 1))*2 - 1
random_img = random_img.astype(np.float32)
random_img = random_img.transpose((0,3,1,2))

In [3]:
from torchnetjson.builder import build_net

def make_model(
    *,
    n_timesteps,
    acc_mode,
    pooling_ksize,
    debug_args=None,
):
    model_json = gen_maskcnn_polished_with_rcnn_k_bl(
        (50, 50), 3,
        n_timesteps=n_timesteps,
        acc_mode=acc_mode,
        pooling_ksize=pooling_ksize,
        debug_args=debug_args,
    )
    print(json.dumps(model_json, indent=2))
    torch.manual_seed(0)
    model = build_net(model_json)

    with no_grad():
        for x, y in model.named_parameters():
            print(x, y.size(), y.mean(), y.std())

    return model




In [4]:
from torch.backends import cudnn
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# let's put some test data.
def eval_model(model_to_eval, check=True):
    model_to_eval.cuda().eval()

    with no_grad():
        out_this = model_to_eval(tensor(random_img).cuda())
        if check:
            assert type(out_this) is tuple and len(out_this) == 1
    
    if check:
        print(out_this[0].size())
        return out_this[0].detach().cpu().numpy()
    else:
        return out_this


def check_ff():
    model_a = make_model(
        n_timesteps=1,
        acc_mode='instant',
        pooling_ksize=2,
    )
    model_b = make_model(
        n_timesteps=1,
        acc_mode='cummean',
        pooling_ksize=2,
    )
    out_a = eval_model(model_a)
    out_b = eval_model(model_b)
    
    assert np.array_equal(out_a, out_b)

    
def check_instant_vs_cummean():
    model_a = make_model(
        n_timesteps=8,
        acc_mode='instant',
        pooling_ksize=1,
        debug_args={'only_fc': True}
    )
    model_b = make_model(
        n_timesteps=8,
        acc_mode='cummean',
        pooling_ksize=1,
        debug_args={'only_fc': True}
    )
    out_a = eval_model(model_a)
    out_b = eval_model(model_b)
    
    assert out_a.shape == out_b.shape == (8, 10, 3)
    
    for idx in range(8):
        instant_out = np.mean(out_a[:idx+1], axis=0)
        cummean_out = out_b[idx]
        assert instant_out.shape == cummean_out.shape == (10,3)
        print(instant_out.mean(), instant_out.std())
        print(cummean_out.mean(), cummean_out.std())
        assert np.allclose(instant_out, cummean_out, atol=1e-5)

def check_stack_vs_no_stack():
    model_a = make_model(
        n_timesteps=8,
        acc_mode='cummean',
        pooling_ksize=2,
    )
    model_b = make_model(
        n_timesteps=8,
        acc_mode='cummean',
        pooling_ksize=2,
        debug_args={'stack': False}
    )
    
    out_a = eval_model(model_a, check=False)
    out_b = eval_model(model_b, check=False)
    out_a = out_a[0].detach().cpu().numpy()
    out_b = [x.detach().cpu().numpy() for x in out_b]
    assert out_a.shape == (8, 10, 3)
    assert np.array_equal(out_a, np.asarray(out_b))

In [5]:
check_ff()

{'pooling', 'final_act', 'bn_output', 'fc'}
{
  "module_dict": {
    "bn_input": {
      "name": "torch.nn.batchnorm2d",
      "params": {
        "num_features": 1,
        "eps": 0.001,
        "momentum": 0.1,
        "affine": true
      },
      "init": {}
    },
    "bl_stack": {
      "name": "rcnn_kriegeskorte.blstack",
      "params": {
        "n_timesteps": 1,
        "channel_list": [
          1,
          48,
          48,
          48
        ],
        "ksize_list": [
          13,
          3,
          3
        ],
        "pool_ksize": 1,
        "act_fn": "softplus"
      },
      "init": {
        "strategy": "normal",
        "parameters": {
          "std": 0.01
        }
      }
    },
    "accumulator": {
      "name": "rcnn_kriegeskorte.accumulator",
      "params": {
        "mode": "instant"
      },
      "init": null
    },
    "pooling": {
      "name": "torch.nn.maxpool2d",
      "params": {
        "kernel_size": 2,
        "ceil_mode": true
      },
  

torch.Size([1, 10, 3])
torch.Size([1, 10, 3])


In [6]:
check_instant_vs_cummean()

{'fc'}
{
  "module_dict": {
    "bn_input": {
      "name": "torch.nn.batchnorm2d",
      "params": {
        "num_features": 1,
        "eps": 0.001,
        "momentum": 0.1,
        "affine": true
      },
      "init": {}
    },
    "bl_stack": {
      "name": "rcnn_kriegeskorte.blstack",
      "params": {
        "n_timesteps": 8,
        "channel_list": [
          1,
          48,
          48,
          48
        ],
        "ksize_list": [
          13,
          3,
          3
        ],
        "pool_ksize": 1,
        "act_fn": "softplus"
      },
      "init": {
        "strategy": "normal",
        "parameters": {
          "std": 0.01
        }
      }
    },
    "accumulator": {
      "name": "rcnn_kriegeskorte.accumulator",
      "params": {
        "mode": "instant"
      },
      "init": null
    },
    "pooling": {
      "name": "torch.nn.maxpool2d",
      "params": {
        "kernel_size": 1,
        "ceil_mode": true
      },
      "init": null
    },
    "fc": {
 

In [7]:
check_stack_vs_no_stack()

{'pooling', 'final_act', 'bn_output', 'fc'}
{
  "module_dict": {
    "bn_input": {
      "name": "torch.nn.batchnorm2d",
      "params": {
        "num_features": 1,
        "eps": 0.001,
        "momentum": 0.1,
        "affine": true
      },
      "init": {}
    },
    "bl_stack": {
      "name": "rcnn_kriegeskorte.blstack",
      "params": {
        "n_timesteps": 8,
        "channel_list": [
          1,
          48,
          48,
          48
        ],
        "ksize_list": [
          13,
          3,
          3
        ],
        "pool_ksize": 1,
        "act_fn": "softplus"
      },
      "init": {
        "strategy": "normal",
        "parameters": {
          "std": 0.01
        }
      }
    },
    "accumulator": {
      "name": "rcnn_kriegeskorte.accumulator",
      "params": {
        "mode": "cummean"
      },
      "init": null
    },
    "pooling": {
      "name": "torch.nn.maxpool2d",
      "params": {
        "kernel_size": 2,
        "ceil_mode": true
      },
  